# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,midvagur,62.0511,-7.1939,-3.54,74,66,1.03,FO,1707118468
1,1,walvis bay,-22.9575,14.5053,18.94,75,33,2.74,NaN,1707118468
2,2,orsa,54.5153,30.4053,-1.40,93,100,4.46,BY,1707118469
3,3,hamilton,39.1834,-84.5333,3.21,71,0,7.20,US,1707118435
4,4,blackmans bay,-43.0167,147.3167,21.07,45,33,1.34,AU,1707118469


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng", "Lat", geo = True,
    tiles = "OSM", frame_width = 700, 
    frame_height = 500, size = "Humidity", 
    color = "City")

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_cities = city_data_df[(city_data_df["Cloudiness"]<5) & (city_data_df["Max Temp"]<25) 
                                    & (city_data_df["Max Temp"]>15) & (city_data_df["Wind Speed"]<5)]

# Drop any rows with null values
ideal_weather_cities.dropna

# Display sample data
ideal_weather_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
91,91,cabo san lucas,22.8909,-109.9124,20.28,72,0,0.45,MX,1707118344
100,100,machali,-34.1833,-70.6667,16.46,69,0,0.79,CL,1707118490
109,109,gisborne,-38.6533,178.0042,17.79,74,0,0.92,NZ,1707118318
110,110,acapulco de juarez,16.8634,-99.8901,22.90,83,0,2.57,MX,1707118492
160,160,cidade velha,14.9167,-23.6167,20.26,88,0,3.60,CV,1707118504
182,182,puerto penasco,31.3167,-113.5333,17.08,66,0,3.91,MX,1707118509
200,200,lazaro cardenas,17.9583,-102.2000,20.58,75,0,1.68,MX,1707118513
205,205,kruisfontein,-34.0033,24.7314,23.82,56,1,4.79,ZA,1707118515
298,298,al wafrah,28.6392,47.9306,16.63,34,3,1.11,KW,1707118536
322,322,mount gambier,-37.8333,140.7667,17.27,47,0,4.71,AU,1707118519


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_cities[["City", "Lat", "Lng", "Humidity", "Country"]].copy(deep=True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
91,cabo san lucas,22.8909,-109.9124,72,MX,
100,machali,-34.1833,-70.6667,69,CL,
109,gisborne,-38.6533,178.0042,74,NZ,
110,acapulco de juarez,16.8634,-99.8901,83,MX,
160,cidade velha,14.9167,-23.6167,88,CV,
182,puerto penasco,31.3167,-113.5333,66,MX,
200,lazaro cardenas,17.9583,-102.2000,75,MX,
205,kruisfontein,-34.0033,24.7314,56,ZA,
298,al wafrah,28.6392,47.9306,34,KW,
322,mount gambier,-37.8333,140.7667,47,AU,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cabo san lucas - nearest hotel: Comfort Rooms
machali - nearest hotel: Hotel Boutique Il Giardino
gisborne - nearest hotel: Portside Hotel
acapulco de juarez - nearest hotel: Hotel del Valle
cidade velha - nearest hotel: Encosta Azul
puerto penasco - nearest hotel: Hotel Paraíso Del Desierto
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
kruisfontein - nearest hotel: No hotel found
al wafrah - nearest hotel: No hotel found
mount gambier - nearest hotel: The Old Mount Gambier Gaol
nova sintra - nearest hotel: Residência Ka Dencho
san luis de la loma - nearest hotel: No hotel found
jalpan - nearest hotel: Posada Karina
mar del plata - nearest hotel: Gran hotel Mónaco
tomatlan - nearest hotel: Hotel Hacienda Vieja Tomatlan
la'ie - nearest hotel: No hotel found
tocopilla - nearest hotel: hotel Isidora
brandon - nearest hotel: Brandon Motor Lodge
castillos - nearest hotel: A mi gente


,City,Lat,Lng,Humidity,Country,Hotel Name
91,cabo san lucas,22.8909,-109.9124,72,MX,Comfort Rooms
100,machali,-34.1833,-70.6667,69,CL,Hotel Boutique Il Giardino
109,gisborne,-38.6533,178.0042,74,NZ,Portside Hotel
110,acapulco de juarez,16.8634,-99.8901,83,MX,Hotel del Valle
160,cidade velha,14.9167,-23.6167,88,CV,Encosta Azul
182,puerto penasco,31.3167,-113.5333,66,MX,Hotel Paraíso Del Desierto
200,lazaro cardenas,17.9583,-102.2000,75,MX,Hotel Sol del Pacífico
205,kruisfontein,-34.0033,24.7314,56,ZA,No hotel found
298,al wafrah,28.6392,47.9306,34,KW,No hotel found
322,mount gambier,-37.8333,140.7667,47,AU,The Old Mount Gambier Gaol


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng", "Lat", geo = True,
    tiles = "OSM", frame_width = 700, 
    frame_height = 500, size = "Humidity", 
    color = "City", hover_cols=["Hotel Name", "Country"])

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)